<a href="https://colab.research.google.com/github/RevHeadGuy/Comic_Panel_Creator/blob/main/Comic_Panel_Creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai gradio requests pillow fpdf gTTS

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.3 MB/s eta 0:00:00
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=674bd57f0781ac092d33f42e6f3569d582f068bf192074a677ed9c6700db771a
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [ ]:
!pip uninstall -y openai
!pip install openai==0.28

Found existing installation: openai 1.86.0
Uninstalling openai-1.86.0:
  Successfully uninstalled openai-1.86.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.4 MB/s eta 0:00:00


In [ ]:
import openai
import requests
import gradio as gr
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
from fpdf import FPDF
from gtts import gTTS
import tempfile

GROQ_API_KEY = "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"
UNSPLASH_ACCESS_KEY = "YiUpa-8y2KlPaMxUwPDvma5MJZZ6oc5S-pFvxQLQWO8"

# Setting up Groq (OpenAI-compatible)
openai.api_key = GROQ_API_KEY
openai.api_base = "https://api.groq.com/openai/v1"


In [ ]:
def generate_script(prompt, style, characters, num_panels):
    try:
        char_str = ", ".join(characters)
        messages = [
            {"role": "system", "content": f"You are a comic script writer. Write a {num_panels}-panel script in {style} style. Characters: {char_str}. Use Panel 1:, Panel 2: format."},
            {"role": "user", "content": prompt}
        ]
        response = openai.ChatCompletion.create(
            model="llama3-70b-8192",
            messages=messages,
            max_tokens=900,
            temperature=0.9
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        return f"⚠️ Error generating script: {str(e)}"


In [ ]:
def get_image(prompt):
    try:
        query = prompt.split()[0]
        url = f"https://api.unsplash.com/photos/random?query={query}&client_id={UNSPLASH_ACCESS_KEY}"
        r = requests.get(url).json()
        return r.get("urls", {}).get("regular", "https://via.placeholder.com/600x400.png?text=No+Image")
    except Exception:
        return "https://via.placeholder.com/600x400.png?text=Image+Error"

In [ ]:
def draw_bubbles(image_url, panel_text):
    try:
        r = requests.get(image_url)
        img = Image.open(BytesIO(r.content)).convert("RGBA")
        draw = ImageDraw.Draw(img)
        draw.rectangle([(0, 0), img.size], outline=(255, 165, 0), width=10)

        font = ImageFont.load_default()
        lines = panel_text.split("\n")
        y = 10
        for line in lines:
            draw.rectangle([(10, y), (590, y+25)], fill=(255, 165, 0, 200))
            draw.text((15, y), line[:80], font=font, fill=(0, 0, 0))
            y += 30
        path = tempfile.mktemp(suffix=".png")
        img.save(path)
        return path
    except Exception as e:
        print("Error drawing bubble:", e)

In [ ]:
def text_to_speech(text):
    try:
        tts = gTTS(text)
        audio_path = tempfile.mktemp(suffix=".mp3")
        tts.save(audio_path)
        return audio_path
    except Exception:
        return None


In [ ]:
def make_pdf(images, script):
    try:
        pdf = FPDF()
        pdf.set_auto_page_break(auto=True)
        pdf.set_font("Arial", size=12)
        pdf.add_page()
        pdf.multi_cell(0, 10, script)
        for img_path in images:
            pdf.add_page()
            pdf.image(img_path, x=10, y=30, w=180)
        out_path = tempfile.mktemp(suffix=".pdf")
        pdf.output(out_path)
        return out_path
    except Exception:
        return None

In [ ]:
def generate_comic(prompt, style, char_input, panels):
    try:
        characters = [c.strip() for c in char_input.split(",") if c.strip()]
        if not characters or not prompt:
            return "Characters or prompt missing.", None, None, None

        script = generate_script(prompt, style, characters, panels)
        panel_texts = script.split("Panel ")[1:]

        images = []
        for pt in panel_texts[:panels]:
            text = pt.split(":", 1)[1].strip()
            img_url = get_image(prompt)
            img_path = draw_bubbles(img_url, text)
            images.append(img_path)

        audio = text_to_speech(script)
        pdf = make_pdf(images, script)

        return script, images[0] if images else None, audio, pdf
    except Exception as e:
        return f"Error: {str(e)}", None, None, None

In [ ]:
with gr.Blocks(css="""
    body { background-color: #111; color: #eee; font-family: Comic Sans MS; }
""") as demo:
    gr.Markdown("<h2 style='text-align: center;'> 🎨 AI Comic Panel Generator</h2>")
    scene = gr.Textbox(label="Scene Prompt", placeholder="e.g. A robot meets a wizard")
    style = gr.Dropdown(["funny", "dramatic", "sci-fi", "romantic", "noir", "fantasy", "superhero",
     "horror", "mystery", "slice-of-life", "sarcastic", "absurdist",
     "historical", "apocalyptic", "western", "cyberpunk", "steampunk",
     "wholesome", "anime", "chibi", "retro", "minimalist"], value="funny", label="Comic Style")
    characters = gr.Textbox(label="Characters (comma-separated)", placeholder="e.g. Max, Luna")
    panels = gr.Slider(1, 10, step=1, label="Number of Panels", value=2)
    btn = gr.Button("Generate Comic")

    out_script = gr.Textbox(label="Comic Script", lines=10)
    out_img = gr.Image(label="First Panel Image")
    out_audio = gr.Audio(label="Voice Narration")
    out_pdf = gr.File(label="Download PDF")

    btn.click(fn=generate_comic,
              inputs=[scene, style, characters, panels],
              outputs=[out_script, out_img, out_audio, out_pdf])

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b4db9d01059a29b9e2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
